In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import mysql.connector

def connect_db():
    conn = mysql.connector.connect(
        host='localhost',
        user='root',
        password='10052022', 
        database='LibraryManagement'
    )
    return conn

def view_books():
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("SELECT Title, ISBN, Quantity FROM Books")
    books = cursor.fetchall()
    conn.close()

    books_window = tk.Toplevel(root)
    books_window.title("All Books")
    books_window.geometry("400x300")

    tree = ttk.Treeview(books_window, columns=("Title", "ISBN", "Quantity"), show="headings")
    tree.heading("Title", text="Title")
    tree.heading("ISBN", text="ISBN")
    tree.heading("Quantity", text="Quantity")
    tree.pack(fill=tk.BOTH, expand=True)

    for book in books:
        tree.insert("", tk.END, values=book)

# Borrow a book function
def borrow_book():
    def submit_borrow():
        member_id = member_id_entry.get()
        book_id = book_id_entry.get()

        conn = connect_db()
        cursor = conn.cursor()

        cursor.execute("SELECT Quantity FROM Books WHERE BookID = %s", (book_id,))
        result = cursor.fetchone()
        if result and result[0] > 0:
            cursor.execute(
                "INSERT INTO Loans (MemberID, BookID, DueDate) VALUES (%s, %s, DATE_ADD(CURRENT_DATE, INTERVAL 14 DAY))",
                (member_id, book_id)
            )
            cursor.execute("UPDATE Books SET Quantity = Quantity - 1 WHERE BookID = %s", (book_id,))
            conn.commit()
            messagebox.showinfo("Success", "Book borrowed successfully!")
        else:
            messagebox.showerror("Error", "Book is not available.")
        
        conn.close()
        borrow_window.destroy()

    borrow_window = tk.Toplevel(root)
    borrow_window.title("Borrow Book")
    borrow_window.geometry("300x200")

    tk.Label(borrow_window, text="Member ID:").pack(pady=5)
    member_id_entry = tk.Entry(borrow_window)
    member_id_entry.pack(pady=5)

    tk.Label(borrow_window, text="Book ID:").pack(pady=5)
    book_id_entry = tk.Entry(borrow_window)
    book_id_entry.pack(pady=5)

    tk.Button(borrow_window, text="Submit", command=submit_borrow).pack(pady=10)

# Return a book function
def return_book():
    def submit_return():
        loan_id = loan_id_entry.get()

        conn = connect_db()
        cursor = conn.cursor()

        cursor.execute("SELECT BookID FROM Loans WHERE LoanID = %s AND Status = 'loaned'", (loan_id,))
        result = cursor.fetchone()
        if result:
            book_id = result[0]
            cursor.execute("UPDATE Loans SET ReturnDate = CURRENT_DATE, Status = 'returned' WHERE LoanID = %s", (loan_id,))
            cursor.execute("UPDATE Books SET Quantity = Quantity + 1 WHERE BookID = %s", (book_id,))
            conn.commit()
            messagebox.showinfo("Success", "Book returned successfully!")
        else:
            messagebox.showerror("Error", "Loan not found or already returned.")

        conn.close()
        return_window.destroy()

    return_window = tk.Toplevel(root)
    return_window.title("Return Book")
    return_window.geometry("300x200")

    tk.Label(return_window, text="Loan ID:").pack(pady=5)
    loan_id_entry = tk.Entry(return_window)
    loan_id_entry.pack(pady=5)

    tk.Button(return_window, text="Submit", command=submit_return).pack(pady=10)

# View overdue books function
def view_overdue_books():
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("""
        SELECT l.LoanID, m.FirstName, m.LastName, b.Title, l.DueDate 
        FROM Loans l 
        JOIN Members m ON l.MemberID = m.MemberID 
        JOIN Books b ON l.BookID = b.BookID 
        WHERE l.DueDate < CURRENT_DATE AND l.Status = 'loaned'
    """)
    overdue_books = cursor.fetchall()
    conn.close()

    overdue_window = tk.Toplevel(root)
    overdue_window.title("Overdue Books")
    overdue_window.geometry("400x300")

    tree = ttk.Treeview(overdue_window, columns=("LoanID", "Member", "Title", "DueDate"), show="headings")
    tree.heading("LoanID", text="Loan ID")
    tree.heading("Member", text="Member")
    tree.heading("Title", text="Book Title")
    tree.heading("DueDate", text="Due Date")
    tree.pack(fill=tk.BOTH, expand=True)

    for row in overdue_books:
        tree.insert("", tk.END, values=(row[0], f"{row[1]} {row[2]}", row[3], row[4]))

# Create main window
root = tk.Tk()
root.title("Library Management System")
root.geometry("600x400")

# Add Labels and Buttons
title_label = tk.Label(root, text="Library Management", font=("Arial", 20))
title_label.pack(pady=10)

view_books_button = tk.Button(root, text="View All Books", command=view_books)
view_books_button.pack(pady=10)

borrow_book_button = tk.Button(root, text="Borrow Book", command=borrow_book)
borrow_book_button.pack(pady=10)

return_book_button = tk.Button(root, text="Return Book", command=return_book)
return_book_button.pack(pady=10)

view_overdue_books_button = tk.Button(root, text="View Overdue Books", command=view_overdue_books)
view_overdue_books_button.pack(pady=10)

# Start the Tkinter event loop
root.mainloop()
